In [4]:
# to prevent to restart kernel when any changes are made to any imported file
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# to import any file from some other directory

# to stop printing warnings
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn
    
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_colwidth', -1)

# to increase cells width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# to enable collapsible Headings and Functions
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter nbextensions_configurator enable --user
# !jupyter nbextension enable codefolding/main
# search collapsible to enable

# enable dark theme
# !pip install jupyterthemes
# !jt -t monokai

# monokai
# solarizedd
# !jt -r
import numpy as np
import pandas as pd
import sys, os
from datetime import datetime, timedelta
import requests
import json
import timeit
from datetime import datetime
import os.path, time
from datetime import datetime, timedelta
from threading import Timer
import pdb, random

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import fasttext
import string
from scipy.stats import hmean
from sklearn.model_selection import TimeSeriesSplit
from random import seed
from random import random
from random import randrange
from sklearn.model_selection import train_test_split

In [5]:
!pwd

PATH="/home/cloud_user/faisal/experiments/tweets_sentiments/"

/home/cloud_user/faisal/experiments/100_day_ml


In [6]:
!ls {PATH}

Tweets.csv


# Get Data

In [7]:
data=pd.read_csv(f"{PATH}Tweets.csv")
print(data.shape)

data.head(2)

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)


In [8]:
df=data.reset_index(drop=True).copy()
df.tweet_created=pd.to_datetime(df.tweet_created)
df=df.rename(columns={"airline_sentiment": "target", "tweet_created": "date"})

df.shape

(14640, 15)

# Util Functions

In [9]:
def preprocess(mydf, Xcol, ycol):
    mydf[Xcol] = mydf[Xcol].apply(lambda x: ((x.encode("unicode_escape").decode("utf-8"))))
    mydf[Xcol] = mydf[Xcol].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))
    mydf[Xcol] = mydf[Xcol].str.lower().str.replace('\n',' ')
    mydf[ycol] = mydf[ycol].astype(str)
    return mydf

In [10]:
def save_file(txts, fname):
    f = open(fname,'w')
    f.write('\n'.join(txts))
    f.close()
    
    return True

In [11]:
def to_fasttext_syntax(mydf, Xcol, ycol, fasttext_txt):
    mydf[fasttext_txt] = '__label__'+mydf[ycol]+' '+mydf[Xcol]
    return mydf

In [12]:
def split_random(mydf, Xcol, ycol, fasttext_txt):
    df=preprocess(mydf, Xcol, ycol)
    trn_df, vld_df, ts_df = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    trn_df, vld_df=to_fasttext_syntax(trn_df, Xcol, ycol, fasttext_txt), to_fasttext_syntax(vld_df, Xcol, ycol, fasttext_txt)
    print("Train: ", trn_df.shape); print("Valid: ", vld_df.shape); print("Test: ", ts_df.shape)
    return trn_df, vld_df, ts_df




In [13]:
def split_dt(mydf, Xcol, ycol, fasttext_txt, trn_split, vld_split=None):
    df=preprocess(mydf, Xcol, ycol)
    
    if vld_split:
        trn_df = df.loc[df.date<trn_split]
        vld_df = df.loc[(df.date>=trn_split) & (df.date<=vld_split)]
        ts_df  = df.loc[df.date>vld_split]
        trn_df, vld_df=to_fasttext_syntax(trn_df, Xcol, ycol, fasttext_txt), to_fasttext_syntax(vld_df, Xcol, ycol, fasttext_txt)
        print("Train: ", trn_df.shape); print("Valid: ", vld_df.shape); print("Test: ", ts_df.shape)
        return trn_df, vld_df, ts_df
    else:
        trn_df = df.loc[df.date<trn_split]
        vld_df = df.loc[(df.date>=trn_split)]
        trn_df=to_fasttext_syntax(trn_df, Xcol, ycol, fasttext_txt)
        print("Train: ", trn_df.shape); print("Valid: ", vld_df.shape)
        return trn_df, vld_df




In [14]:
def get_classification_report(y_true, y_pred):
    report=classification_report(y_true, y_pred, output_dict=True)
    report=pd.DataFrame(report).T.round(2)
    report.loc["harmonic avg"] = report.iloc[:-3, :].apply(lambda x: round(hmean(x), 2), axis=0)
    return report


In [15]:
def get_results(test_df, classifier, Xcol, ycol):
    y_pred=[]
    y_true=[]
    for txt, lbl in zip(test_df[Xcol], test_df[ycol]):
        y_pred.append(classifier.predict([txt])[0][0][0][9:])
        y_true.append(lbl)

    y_pred = np.array(y_pred, dtype='O')
    y_true = np.array(y_true, dtype='O')
    conf_mat=confusion_matrix(y_true, y_pred)
    cls_rpt=get_classification_report(y_true, y_pred)
    print(conf_mat)
    print("\n")
    print(cls_rpt)
    print(f"\n =================== \n")
    return conf_mat, cls_rpt
    

In [16]:
def classify_fasttext(trn_df, vld_df, ts_df, fasttext_txt, autotune_label):
    results={}
    
    trn_fn, vld_fn = "/tmp/df.train", "/tmp/df.valid"
    
    _, _=save_file(trn_df, trn_fn, fasttext_txt), save_file(vld_df, vld_fn, fasttext_txt)
    
    classifier1 = fasttext.train_supervised(trn_fn, loss='softmax')
    print("\nclassifier1: ")
    conf_mat1, cls_rpt1 = get_results(ts_df, classifier1, Xcol, ycol)
    results["classifier1"]=(classifier1,conf_mat1, cls_rpt1)
    
    classifier2 = fasttext.train_supervised(input=trn_fn, autotuneValidationFile=vld_fn, autotuneMetric=f"f1:__label__{autotune_label}")
    print("classifier2: ")
    conf_mat2, cls_rpt2 = get_results(ts_df, classifier2, Xcol, ycol)
    results["classifier2"]=(classifier2,onf_mat2, cls_rpt2)

    classifier3 = fasttext.train_supervised(input=trn_fn, autotuneValidationFile=vld_fn, autotuneDuration=60)
    print("classifier3: ")
    conf_mat3, cls_rpt3 = get_results(ts_df, classifier3, Xcol, ycol)
    results["classifier3"]=(classifier3,conf_mat3, cls_rpt3)
    
    return results


# Classification

In [17]:
Xcol="text"
ycol="target"
fasttext_txt="ft_txt"

# random split
# trn_df, vld_df, ts_df=split_random(df, Xcol, ycol, fasttext_txt)

# split on date, trn, vld, ts
# trn_split="2019-08-31";vld_split="2019-10-31"
# trn_df, vld_df, ts_df=split_dt(mydf, Xcol, ycol, fasttext_txt, trn_split, vld_split)

# split on date, trn, ts
split="2015-02-23"
trn_df, vld_df=split_dt(df, Xcol, ycol, fasttext_txt, split)




Train:  (10268, 16)
Valid:  (4372, 15)


In [ ]:
# results = classify_fasttext(trn_df, vld_df, ts_df, fasttext_txt, "neutral")

## Analysis

In [18]:
def subsample(df, Xcol, ycol, ratio=0.60):
    X_train, _, _, _ = train_test_split(df[Xcol], df[ycol], test_size=ratio, shuffle=True, stratify=df[ycol], random_state=42)
    
    return list(X_train)
    

In [19]:
def train_bag(df, fasttext_txt, ycol):
    trn_X_sample=subsample(df, fasttext_txt, ycol)
    trn_fn = "tmpdir/df_sample.train"
    _=save_file(trn_X_sample, trn_fn)
    
    
    classifier = fasttext.train_supervised(trn_fn, loss='softmax')
    return classifier
    

In [20]:
def get_preds_bag(classifier, vld_df, Xcol, ycol, i):
    vld_df[f"pred_{i}"] = vld_df[Xcol].apply(lambda x: classifier.predict([x])[0][0][0][9:])
    vld_df[f"pred_{i}_correct"]=np.where(vld_df[f"pred_{i}"]==vld_df.target, True, False)
    return vld_df


In [21]:
def get_preds(classifier, vld_df, Xcol, ycol):
    vld_df[f"pred"] = vld_df[Xcol].apply(lambda x: classifier.predict([x])[0][0][0][9:])
    vld_df[f"pred_correct"]=np.where(vld_df[f"pred"]==vld_df.target, True, False)
    return vld_df



### 1. Timeseries

In [22]:
def train_timeseries(df, Xcol, ycol, fasttext_txt, n_splits=5):
    df=to_fasttext_syntax(df, Xcol, ycol, fasttext_txt)
    
    vld_df_merged=pd.DataFrame()

    tscv = TimeSeriesSplit(n_splits=n_splits)
    for trn_idx, vld_idx in tscv.split(df):

        trn_df, vld_df=df.iloc[trn_idx],df.iloc[vld_idx]
        classifier=train_bag(trn_df, fasttext_txt, ycol)

        vld_df=get_preds(classifier, vld_df, Xcol, ycol)
        vld_df_merged=pd.concat([vld_df_merged, vld_df], axis=0)

    return vld_df_merged


In [23]:
def get_analysis(vld_df_merged, cols):
    vld_counts=vld_df_merged[cols[0]].value_counts().to_dict()
    vld_df_merged["val_counts"]=vld_df_merged[cols[0]].map(vld_counts)

    grp=pd.DataFrame(vld_df_merged.groupby(cols).size()).rename(columns={0: "counts"})
    grp["ratios"]=(grp.counts / grp.index.get_level_values(1)).round(2)
    return grp



In [24]:
ts_result=train_timeseries(df, Xcol, ycol, fasttext_txt, 5)

In [26]:
grp=get_analysis(ts_result, ["airline", "val_counts","pred_correct"])
grp

counts  ratios
airline    val_counts pred_correct                
American   2759       False         549     0.20  
                      True          2210    0.80  
Delta      2222       False         851     0.38  
                      True          1371    0.62  
Southwest  2420       False         1012    0.42  
                      True          1408    0.58  
US Airways 2913       False         516     0.18  
                      True          2397    0.82  
United     1886       False         615     0.33  
                      True          1271    0.67

In [29]:
grp.groupby(level=0).apply(lambda x: hmean(x.ratios)).sort_values(ascending=False)


airline
Southwest     0.4872
Delta         0.4712
United        0.4422
American      0.3200
US Airways    0.2952
dtype: float64

### 2. Bagging

In [30]:
Xcol="text"
ycol="target"
fasttext_txt="ft_txt"

split="2015-02-23"
trn_df, vld_df=split_dt(df, Xcol, ycol, fasttext_txt, split)



Train:  (10268, 16)
Valid:  (4372, 16)


In [31]:
def train_bagging(trn_df, vld_df, Xcol, ycol, fasttext_txt, kfolds=5):

    for i in range(1,kfolds+1):
        classifier=train_bag(trn_df, fasttext_txt, ycol)
        vld_df=get_preds_bag(classifier, vld_df, Xcol, ycol, i)

    return vld_df

In [32]:
def get_bagging_analysis(vld_df, cols, n_folds):
    vld_df["n_correct"]=vld_df[[f"pred_{i}_correct" for i in range(1,n_folds+1)]].sum(axis=1)
    vld_df["n_incorrect"]=n_folds-vld_df.n_correct
    
    vld_counts=vld_df[cols[0]].value_counts().to_dict()
    vld_df["val_counts"]=vld_df[cols[0]].map(vld_counts)

    grp=pd.DataFrame(vld_df[cols].groupby(cols).size()).rename(columns={0:"counts"})
    grp["ratios"]=(grp.counts / grp.index.get_level_values(1)).round(2)

    return grp

In [33]:
kfolds=5
bag_results=train_bagging(trn_df, vld_df, Xcol, ycol, fasttext_txt, kfolds)

cols=["airline", "val_counts", "n_incorrect"]
grp=get_bagging_analysis(vld_df, cols, kfolds)
grp

counts  ratios
airline        val_counts n_incorrect                
American       1767       0            1360    0.77  
                          1            12      0.01  
                          2            7       0.00  
                          3            3       0.00  
                          4            7       0.00  
                          5            378     0.21  
Delta          504        0            310     0.62  
                          1            6       0.01  
                          2            9       0.02  
                          3            1       0.00  
                          4            6       0.01  
                          5            172     0.34  
Southwest      459        0            312     0.68  
                          1            5       0.01  
                          2            3       0.01  
                          3            1       0.00  
                          4            1       0.00  
                          5            137     0.30  
US Airways     665        0            518     0.78  
                          1            1       0.00  
                          2            1       0.00  
                          3            2       0.00  
                          4            2       0.00  
                          5            141     0.21  
United         857        0            649     0.76  
                          1            1       0.00  
                          2            2       0.00  
                          3            3       0.00  
                          4            1       0.00  
                          5            201     0.23  
Virgin America 120        0            70      0.58  
                          1            2       0.02  
                          3            1       0.01  
                          5            47      0.39